In [74]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.nasnet import NASNetMobile
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
from matplotlib.pyplot import plot as plt

### Prepare Dataset

In [92]:
TRAIN_DIR = "../../data/SantaCruzIsland_Labeled_5Class"
BATCH_SIZE = 8
NUM_EPOCHES = 1
WIDTH = 1920
HEIGHT = 1080
num_train_images = 100

In [34]:
train_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=90, \
                                         horizontal_flip=True, vertical_flip=True)

In [43]:
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(WIDTH, HEIGHT), batch_size = BATCH_SIZE)

Found 1554 images belonging to 5 classes.


### Create Model

In [32]:
resnet50_model = ResNet50(input_shape=(WIDTH, HEIGTH, 3), weights='imagenet', include_top=False)

In [25]:
nasnet_mobile = NASNetMobile(include_top=False, weights='imagenet', pooling='max')

19996672/19993432 [==============================] - 13s 1us/step


In [85]:
def build_model(base_model, dropout, fc_layers, num_classes):

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # let's add fully-connected layers
    # Every FC layer has a dropout probability
    for fc in fc_layers:
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)

    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)    
    
    return model

In [86]:
class_list = ["Birds", "Empty", "Fox", "Humans", "Rodents"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

layered_model = build_model(resnet50_model, dropout=dropout, fc_layers=FC_LAYERS, num_classes=len(class_list))

### Train the model

In [87]:
def train_model(model, model_name):

    # create adam optimizer with learning rate
    adam = Adam(lr=0.00001)

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    
    checkpoint = ModelCheckpoint("./checkpoints/{0}_model_weights.h5".format(model_name), monitor=["acc"], verbose=1, mode='max')
    
    history = model.fit_generator(train_generator, epochs = NUM_EPOCHES, workers=8, 
                                  steps_per_epoch=num_train_images // BATCH_SIZE, 
                                  shuffle=True, callbacks=[checkpoint])

    return history
    
    

In [75]:
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    
    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    plt.figure()
    plt.plot(epochs, loss, 'r.')
    plt.plot(epochs, val_loss, 'r')
    plt.title('Training and validation loss')
    plt.show()
    
    plt.savefig('training_history.png')
    

In [93]:
history = train_model(layered_model, "ResNet50")

Epoch 1/1
12/12 [==============================] - 502s 42s/step - loss: 1.4028 - acc: 0.4271

Epoch 00001: saving model to ./checkpoints/ResNet50_model_weights.h5


OSError: Unable to create file (unable to open file: name = './checkpoints/ResNet50_model_weights.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)